Dans cet exercice, nous allons utiliser le service Form Recognizer pour :

- Former et tester un modèle personnalisé à l’aide d’une formation non supervisée (sans étiquettes) avec le SDK Python ou .NET.
- Former et tester un modèle personnalisé à l’aide d’une formation supervisée (avec étiquettes) avec le SDK Python ou .NET.

## Pour commencer

Pour utiliser le service Form Recognizer, vous devez récupérer la ressource Form Recognizer dans votre souscription à Azure. Vous devez utiliser le portail Azure pour créer cette ressource.

Une fois que la ressource a été déployée, dirigez-vous vers la page <b>Keys and Endpoint</b>. Vous aurez besoin de l'endpoint et d'une des clés de cette page pour pouvoir accéder au service via le code.

## Rassembler les documents pour l'entraînement

En exemple dans cet exercice, on peut procéder ainsi :
- un premier modèle non supervisé avec uniquement les fichiers .jpg
- un second modèle supervisé avec les fichiers .jpg et .json (les fichiers .json contenant la labellisation des fichiers)

On devra se servir de l'ID de souscription, du nom du groupe de ressource et la localisation du groupe de ressources créé dans un fichier setup. En mettant en route ce fichier setup, on active le groupe de ressource et on obtient en sortie un lien URI SAS (Shared Access Signature) qui donne accès à ce groupe de ressources. Il faudra garder ce lien URI pour plus tard

Sur le portail Azure, une fois la manipulation faite, il faut se rendre au groupe de ressources concerné pour vérifier si les fichiers ont bien été uploadés à partir du repo cloné.

## Entraîner un modèle non supervisé

On va utiliser un SDK Form Recognizer pour entraîner et tester un modèle personnalisé. Pour cela, dans le repo cloné, dans le dossier <b>21-custom-form</b>, si on souhaite travailler en Python, on se rend dans le dossier correspondant.
Pour le dossier train-model, on ouvre un terminal intégré à ce dossier pour insérer la commande suivante :

In [1]:
pip install azure-ai-formrecognizer==3.0.0

Note: you may need to restart the kernel to use updated packages.


Un fichier .env est alors créé. Le fichier doit être édité avec les renseignements suivants :
- l'endpoint de la ressource Form Recognizer
- la clé de la ressource Form Recognizer
- l'URI SAS du conteneur des fichiers nécessaires à l'entraînement

Le fichier train-model.py permettra alors d'entraîner le modèle

In [3]:
python train-model.py

SyntaxError: invalid syntax (<ipython-input-3-e095850bc457>, line 1)

En sortie, il faut récupérer l'ID du modèle ainsi entraîné.

## Tester le modèle non supervisé créé

Maintenant que nous sommes prêts à utiliser notre modèle entraîné à partir des fichiers du container, on va devoir tester notre modèle.

Tout comme pour l'entraînement, il faut aussi lancer un terminal intégré au dossier test-model. Dans ce terminal, on peut alors installer le package Form Recognizer :

In [ ]:
pip install azure-ai-formrecognizer==3.0.0

Dans le dossier test-model, il faut éditer le fichier .env avec les valeurs suivants :
- l'endpoint de la ressource Form Recognizer
- la clé de la ressource Form Recognizer
- l'ID du modèle entraîné

On peut alors mettre en route le script test-model.py. Le formulaire testé, qui se trouve dans le dossier test-model, sera analysé :

In [ ]:
python test-model.py

En output, des scores de confiance sont mis en avant pour les différents champs du formulaire.

## Entraîner un modèle supervisé en utilisant une librairie client

Supposons qu'après avoir entraîné un modèle dédié aux formulaires, on souhaiterait voir comment un modèle entraîné sur des données labellisées se comporte. 
Quand on a entraîné un modèle non supervisé, on a seulement utilisé des formulaires au format .jpg depuis notre container. Maintenant, nous allons entraîner un modèle en utilisant les fichiers .jpg et .json.

Dans le fichier fields.json, on peut vérifier les champs sur lesquels le modèle va s'entraîner pour les formulaires. 

Dans le fichier Form_1.jpg.labels.json, on retrouve la localisation et les valeurs des champs nommés du formulaire.

Dans le fichier Form_1.jpg.ocr.json, on retrouve la disposition textuelle donc la localisation de tous les champs de texte du formulaire.

Dans le dossier train-model, on doit ouvrir le fichier train-model.py et écrire les lignes de code suivantes :

In [ ]:
# Train model
poller = form_training_client.begin_training(trainingDataUrl, use_training_labels=True)
model = poller.result()

Dans le terminal intégré au modèle, on relance le fichier :

In [ ]:
python train-model.py

Un output est alors affiché avec les scores de confiance pour chaque champ. On récupère de ce fait l'ID du modèle.

## Test du modèle supervisé

Dans le dossier test-model, on édite le fichier .env en y insérant l'ID du modèle.

Dans le terminal intégré à test-model, on exécute le fichier :

In [ ]:
python test-model.py

On voit l'output qui peut fournir des noms de champs.